In [4]:
import os 
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="gemma3:12b")

os.environ['SERPAPI_API_KEY'] = '4d11a637c3f2555f052411f12131ee7cbe8025b6aa47a898c9ba9922a2e55811'

In [5]:
from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine" : "google_news", 
    "gl": "KR",
    "hl": "ko",
}
search = SerpAPIWrapper(params=params)

search.run("이차전지 산업")

[{'position': 1,
  'title': '연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜',
  'source': {'name': '경제인뉴스',
   'icon': 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://www.newseconomy.kr&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
  'link': 'https://www.newseconomy.kr/news/articleView.html?idxno=19093',
  'thumbnail': 'https://cdn.newseconomy.kr/news/photo/202601/19093_12777_556.jpg',
  'thumbnail_small': 'https://news.google.com/api/attachments/CC8iK0NnNUJVbmhRU0ZGeWNXWlVhelF3VFJEZ0F4aUFCU2dLTWdhcFpJNnVKUWc',
  'date': '01/08/2026, 01:06 AM, +0000 UTC',
  'iso_date': '2026-01-08T01:06:14Z'},
 {'position': 2,
  'title': '올해 이차전지 충북 기업 수익성 방어 ‘총력’',
  'source': {'name': '중부매일',
   'icon': 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://www.jbnews.com&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
  'link': 'https://www.jbnews.com/news/articleView.html?idxno=1496634',
  'thumbnail': 'https://www.jbnews.com/news/photo/202601/1496634_1343497_

In [6]:
results = search.run("이차전지 산업")
len(results)

100

In [7]:
results[0]

{'position': 1,
 'title': '연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜',
 'source': {'name': '경제인뉴스',
  'icon': 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://www.newseconomy.kr&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
 'link': 'https://www.newseconomy.kr/news/articleView.html?idxno=19093',
 'thumbnail': 'https://cdn.newseconomy.kr/news/photo/202601/19093_12777_556.jpg',
 'thumbnail_small': 'https://news.google.com/api/attachments/CC8iK0NnNUJVbmhRU0ZGeWNXWlVhelF3VFJEZ0F4aUFCU2dLTWdhcFpJNnVKUWc',
 'date': '01/08/2026, 01:06 AM, +0000 UTC',
 'iso_date': '2026-01-08T01:06:14Z'}

In [8]:
results[0].keys()

dict_keys(['position', 'title', 'source', 'link', 'thumbnail', 'thumbnail_small', 'date', 'iso_date'])

In [9]:
url = results[0]['link']
url

'https://www.newseconomy.kr/news/articleView.html?idxno=19093'

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)
docs = loader.load()

docs[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://www.newseconomy.kr/news/articleView.html?idxno=19093', 'title': '연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜ < CEO < 기사본문 - 경제인뉴스', 'description': '이차전지와 전기차 산업이 빠르게 성장하면서, 성능 못지않게 안전 기술의 중요성이 갈수록 부각되고 있다. 특히 화재와 열폭주 문제는 배터리 산업 전반에서 가장 까다롭고 중요한 과제로 꼽힌다. 이러한 흐름 속에서 동방소재㈜는 전기·전자 및 이차전지 산업에 적용되는 안전 소재를 중심으로 기', 'language': 'ko'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜ < CEO < 기사본문 - 경제인뉴스\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n주요서비스 바로가기\n본문 바로가기\n매체정보 바로가기\n로그인 바로가기\n기사검색 바로가기\n전체서비스 바로가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n전체메뉴 버튼\n\n\n\n\n경제\n\n\n\n\nCEO\n\n\n\n\nInterview\n\n\n\n\n사회\n\n\n\n\n라이프\n\n\n\n\n비즈니스\n\n\n\n\n문화\n\n\n\n\n\n\n\n\n\n최종편집\xa02026-01-08 10:11 (목) \n\n\n\n로그인\n회원가입\n\n\n\n\n검색\n\n\n\n\n\n기사검색\n\n검색\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n이전 기

In [11]:
def extract_longest_text(text):
    segments = text.split('\n')
    longest_segment = max(segments, key=len)
    
    return longest_segment

text = docs[0].page_content

longest_text = extract_longest_text(text)
print("가장 긴 텍스트:\n", longest_text)

가장 긴 텍스트:
 초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다변화의 필요성을 인식하던 중 이차전지 산업의 급성장과 함께 배터리 안전 소재 분야로 사업 영역을 확장하게 되었다. 이차전지 화재차단용 불연·단열 시트는 전기차(EV), ESS 등 이차전지를 사용하는 환경에서 열폭주 방지에 효과적인 핵심 안전 소재로, 적용 부위에 따라 불연성·단열성·유연성·완충성 등의 성능을 동시에 구현할 수 있다. 일부 제품은 기존 화재차단 소재로 널리 사용되는 MICA 시트의 대체 소재로도 적합하다.


In [12]:
from tqdm import tqdm
import time 

new_results = []
for result in tqdm(results):
    try:
        url = result['link']
        loader = WebBaseLoader(url)
        docs = loader.load()
        text = docs[0].page_content
        longest_Text = extract_longest_text(text)
        result['content']= longest_text
        new_results.append(result)
    except:
        pass

print("새로운 결과 개수 : ", len(new_results))
new_results[0]


100%|██████████| 100/100 [00:33<00:00,  2.98it/s]

새로운 결과 개수 :  100


{'position': 1,
 'title': '연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜',
 'source': {'name': '경제인뉴스',
  'icon': 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://www.newseconomy.kr&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
 'link': 'https://www.newseconomy.kr/news/articleView.html?idxno=19093',
 'thumbnail': 'https://cdn.newseconomy.kr/news/photo/202601/19093_12777_556.jpg',
 'thumbnail_small': 'https://news.google.com/api/attachments/CC8iK0NnNUJVbmhRU0ZGeWNXWlVhelF3VFJEZ0F4aUFCU2dLTWdhcFpJNnVKUWc',
 'date': '01/08/2026, 01:06 AM, +0000 UTC',
 'iso_date': '2026-01-08T01:06:14Z',
 'content': '초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다변화의 필요성을 인식하던 중 이차전지 산업의 급성장과 함께 배터리 안전 소재 분야로 사업 영역을 확장하게 되었다. 이차전지 화재차단용 불연·단열 시트는 전기차(EV), ESS 등 이차전지를 사용하는 환경에서 열폭주 방지에 효과적인 핵심 안전 소재로, 적용 부위에 따라 불연성·단열성·유연성·완충성 등의 성능을 동시에 구현할 수 있다. 일부 제품은 기존 화재차단 소재로 널리 사용되는 MICA 시트의 대체 소재로도 적합하다.'}

In [13]:
import pandas as pd
data=pd.DataFrame(new_results)
data.head()

,position,title,source,link,thumbnail,thumbnail_small,date,iso_date,content
0,1,"연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜","{'name': '경제인뉴스', 'icon': 'https://encrypted-t...",https://www.newseconomy.kr/news/articleView.ht...,https://cdn.newseconomy.kr/news/photo/202601/1...,https://news.google.com/api/attachments/CC8iK0...,"01/08/2026, 01:06 AM, +0000 UTC",2026-01-08T01:06:14Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다..."
1,2,올해 이차전지 충북 기업 수익성 방어 ‘총력’,"{'name': '중부매일', 'icon': 'https://encrypted-tb...",https://www.jbnews.com/news/articleView.html?i...,https://www.jbnews.com/news/photo/202601/14966...,https://news.google.com/api/attachments/CC8iI0...,"01/07/2026, 07:24 AM, +0000 UTC",2026-01-07T07:24:21Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다..."
2,3,"전극·조립·팩 공정까지…아이비젼웍스, 이차전지 검사 시장 '정조준'","{'name': '핀포인트뉴스', 'icon': 'https://encrypted-...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/photo/2026...,https://news.google.com/api/attachments/CC8iJ0...,"01/08/2026, 04:18 AM, +0000 UTC",2026-01-08T04:18:48Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다..."
3,4,"익산시, 이차전지산업 실무형 기술 인재 양성…산학협력","{'name': '뉴시스', 'icon': 'https://encrypted-tbn...",https://www.newsis.com/view/NISX20260105_00034...,https://image.newsis.com/2026/01/05/NISI202601...,https://news.google.com/api/attachments/CC8iK0...,"01/05/2026, 05:13 AM, +0000 UTC",2026-01-05T05:13:02Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다..."
4,5,"광양시, 2차전지 산업 규제 개선 '속도'","{'name': '뉴스핌', 'icon': 'https://encrypted-tbn...",https://www.newspim.com/news/view/20260107000780,https://img.newspim.com/news/2026/01/07/260107...,https://news.google.com/api/attachments/CC8iMk...,"01/07/2026, 06:20 AM, +0000 UTC",2026-01-07T06:20:00Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다..."


## 뉴스 요약 

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM

from langchain_core.output_parsers import StrOutputParser

#promopt
prompt_template = """다음 내용을 한국어(한글)로 간결하게 요약하라.

규칙:
- 요약 내용만 출력한다
- 영어 문장, 설명, 인사말을 포함하지 않는다
- 불필요한 문구 없이 바로 요약문만 작성한다

내용:
{text}
"""
prompt = PromptTemplate.from_template(prompt_template)

#LLM
llm = OllamaLLM(model="gemma3:12b")

#parser
output_parser = StrOutputParser()

llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'TV 가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장했으며, 화재차단용 불연·단열 시트는 전기차, ESS 등에서 열폭주 방지 및 다양한 성능을 제공하여 MICA 시트 대체 가능성을 지닌다.'

In [22]:
def summarize_news(content):
    response = llm_chain.invoke({"text": content})
    return response

df_test = data.head(3)
df_test['summary'] = df_test['content'].apply(summarize_news)

df_test

C:\Users\Bistelligence\AppData\Local\Temp\ipykernel_10648\2177543294.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['summary'] = df_test['content'].apply(summarize_news)


,position,title,source,link,thumbnail,thumbnail_small,date,iso_date,content,summary
0,1,"연쇄 화재를 지연시키는 이차전지 핵심 기술, 동방소재㈜","{'name': '경제인뉴스', 'icon': 'https://encrypted-t...",https://www.newseconomy.kr/news/articleView.ht...,https://cdn.newseconomy.kr/news/photo/202601/1...,https://news.google.com/api/attachments/CC8iK0...,"01/08/2026, 01:06 AM, +0000 UTC",2026-01-08T01:06:14Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 사업으로 확장했다. 화재차단용 불연·단...
1,2,올해 이차전지 충북 기업 수익성 방어 ‘총력’,"{'name': '중부매일', 'icon': 'https://encrypted-tb...",https://www.jbnews.com/news/articleView.html?i...,https://www.jbnews.com/news/photo/202601/14966...,https://news.google.com/api/attachments/CC8iI0...,"01/07/2026, 07:24 AM, +0000 UTC",2026-01-07T07:24:21Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...","가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장, 화재차단용 불연·..."
2,3,"전극·조립·팩 공정까지…아이비젼웍스, 이차전지 검사 시장 '정조준'","{'name': '핀포인트뉴스', 'icon': 'https://encrypted-...",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/photo/2026...,https://news.google.com/api/attachments/CC8iJ0...,"01/08/2026, 04:18 AM, +0000 UTC",2026-01-08T04:18:48Z,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장했다. 주요 제품은 ...


In [23]:
df_test[['content', 'summary']]

,content,summary
0,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 사업으로 확장했다. 화재차단용 불연·단...
1,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...","가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장, 화재차단용 불연·..."
2,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장했다. 주요 제품은 ...


## 키워드 추출

In [26]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser

#prompt
prompt_template = """Please extract 3 key words form the following content in Korean Hangul(한글) and separate them with commas (,) 
"{text}" 
Key words :"""

prompt = PromptTemplate.from_template(prompt_template)

#LLM 
llm = OllamaLLM(temperature = 0, model="gemma3:12b")

#ouput parser
output_parser = StrOutputParser()

#Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'이차전지, 안전, 필름\n'

In [28]:
# 뉴스 본문을 입력으로 사용하여 핵심 키워드를 추출하는 함수 
def extract_keywords(content):
    response = llm_chain.invoke({"text": content})
    return response

#결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 추출 
df_test['keywords'] = df_test['content'].apply(extract_keywords)

df_test[['content', 'summary' , 'keywords']]

C:\Users\Bistelligence\AppData\Local\Temp\ipykernel_10648\1007077600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords'] = df_test['content'].apply(extract_keywords)


,content,summary,keywords
0,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 사업으로 확장했다. 화재차단용 불연·단...,"이차전지, 불연, 안전"
1,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...","가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장, 화재차단용 불연·...","이차전지, 불연, 안전"
2,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장했다. 주요 제품은 ...,"이차전지, 불연, 안전"


## 뉴스 카테고리 분류

In [31]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser 

#prompt 
prompt_template = """Based on the following content, please classify the news into the approproiate category and provide the category name in Korean without description:
"{text}"
News Category:"""

prompt = PromptTemplate.from_template(prompt_template)

#LLM 
llm = OllamaLLM(temperature = 0, model="gemma3:12b")

#output parser
output_parser = StrOutputParser()

#chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'배터리 소재'

In [33]:
#뉴스 본문을 입력으로 사용하여 카테고리를 분류하는 함수 
def classify_news_category(content):
    response = llm_chain.invoke({"text": content})
    return response

#결과 확인 - 테스트를 우해서 첫 3행만 별도로 추출하여 카테고리 분류 
df_test['category'] = df_test['content'].apply(classify_news_category)

df_test[['content', 'summary', 'keywords', 'category']]

C:\Users\Bistelligence\AppData\Local\Temp\ipykernel_10648\997070006.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['category'] = df_test['content'].apply(classify_news_category)


,content,summary,keywords,category
0,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 사업으로 확장했다. 화재차단용 불연·단...,"이차전지, 불연, 안전",배터리 소재
1,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...","가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장, 화재차단용 불연·...","이차전지, 불연, 안전",배터리 소재
2,"초기에는 TV 등 가전제품에 적용되는 전기절연 필름을 주력으로 공급했으나, 시장 다...",가전제품 필름 제조사에서 이차전지 안전 소재 분야로 사업을 확장했다. 주요 제품은 ...,"이차전지, 불연, 안전",배터리 소재
